In [1]:
!pip install gpflow
!pip install utils
!pip install cartopy
!apt-get -qq install python-cartopy python3-cartopy
!pip uninstall -y shapely    # cartopy and shapely aren't friends (early 2020)

!pip install shapely --no-binary shapely
!pip install eofs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.8/376.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 55.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.6 MB/s eta 0:00:00
  Created wheel for cartopy: filename=Cartopy-0.21.1-cp310-cp310-linux_x86_64.whl size=11102758 sha256=69371189bba2a6ae8addb031cc5e78db545e850930456da7ecfb83703c9a0371
  Stored in directory: /root/.cache/pip/wheels/30/b0/1a/1c1909e00c76653dc4e2ff48555257c0eb2d1698280c8d9955
Successfully built cartopy
E: Unable to locate package python-cartopy
Found existing installation: shapely 2.0.1
Uninstalling shapely-2.0.1:
  Successfully uninstalled shapely-2.0.1
     

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from tensorflow.keras.regularizers import l1_l2
from utils import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

In [3]:
from google.colab import drive


drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob



def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)


def prepare_predictor(data_sets, data_path,time_reindex=True):
    """
    Args:
        data_sets list(str): names of datasets
    """

    # Create training and testing arrays
    if isinstance(data_sets, str):
        data_sets = [data_sets]

    X_all      = []
    length_all = []

    for file in data_sets:
        data = xr.open_dataset(os.path.join(data_path, f"inputs_{file}.nc"))
        X_all.append(data)
        length_all.append(len(data.time))

    X = xr.concat(X_all,dim='time')
    length_all = np.array(length_all)
    # X = xr.concat([xr.open_dataset(data_path + f"inputs_{file}.nc") for file in data_sets], dim='time')
    if time_reindex:
        X = X.assign_coords(time=np.arange(len(X.time)))

    return X, length_all

def prepare_predictand(data_sets,data_path,time_reindex=True):
    if isinstance(data_sets, str):
        data_sets = [data_sets]

    Y_all = []
    length_all = []

    for file in data_sets:
        data = xr.open_dataset(os.path.join(data_path, f"outputs_{file}.nc"))
        Y_all.append(data)
        length_all.append(len(data.time))

    length_all = np.array(length_all)
    Y = xr.concat(Y_all,dim='time').mean('member')
    # Y = xr.concat([xr.open_dataset(data_path + f"outputs_{file}.nc") for file in data_sets], dim='time').mean("member")
    Y = Y.rename({'lon':'longitude','lat': 'latitude'}).transpose('time','latitude', 'longitude').drop(['quantile'])
    if time_reindex:
        Y = Y.assign_coords(time=np.arange(len(Y.time)))

    return Y, length_all


def get_rmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.latitude))
    return np.sqrt(((truth-pred)**2).weighted(weights).mean(['latitude', 'longitude'])).data.mean()

def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean squared error')
    plt.plot(history.epoch, np.array(history.history['loss']),
           label='Train Loss')
    plt.plot(history.epoch, np.array(history.history['val_loss']),
           label = 'Val loss')
    plt.legend()




# Utilities for normalizing the input data
def normalize(data, var, meanstd_dict):
    mean = meanstd_dict[var][0]
    std = meanstd_dict[var][1]
    return (data - mean)/std

def mean_std_plot(data,color,label,ax):

    mean = data.mean(['latitude','longitude'])
    std  = data.std(['latitude','longitude'])
    yr   = data.time.values

    ax.plot(yr,mean,color=color,label=label,linewidth=4)
    ax.fill_between(yr,mean+std,mean-std,facecolor=color,alpha=0.4)

    return yr, mean

In [5]:
cwd = os.getcwd()

train_path = os.path.join(cwd,'/content/gdrive/MyDrive/climate_emulator_duncan_parris/','train_val1')
test_path  = os.path.join(cwd,'/content/gdrive/MyDrive/climate_emulator_duncan_parris/','train_val1')

make_dir(train_path)
make_dir(test_path)

In [6]:
# Training set
train_files = ["historical", "ssp585", "ssp126", "ssp370","hist-GHG","hist-aer"]
X_train_xr, X_length  = prepare_predictor(train_files,train_path)
y_train_xr, y_length  = prepare_predictand(train_files,train_path)

# Test set
X_test_xr, _ = prepare_predictor('ssp245', data_path=test_path,time_reindex=False)
y_test_xr, _ = prepare_predictand('ssp245',data_path=test_path,time_reindex=False)

In [7]:

X_train_df = pd.DataFrame({"CO2": X_train_xr["CO2"].data,
                           "CH4": X_train_xr["CH4"].data
                          }, index=X_train_xr["CO2"].coords['time'].data)

X_test_df  = pd.DataFrame({"CO2": X_test_xr["CO2"].data,
                           "CH4": X_test_xr["CH4"].data
                          }, index=X_test_xr["CO2"].coords['time'].data)

# Normalization
mean, std = X_train_df.mean(), X_train_df.std()

X_train_df   = (X_train_df - mean)/std
X_test_df    = (X_test_df - mean)/std

X_train = X_train_df.to_numpy()
X_test = X_test_df.to_numpy()

print(X_train.shape,X_test.shape)

(753, 2) (86, 2)


In [8]:
var  = 'tas'
y_train = y_train_xr[var].data
y_test  = y_test_xr[var].data

mean = y_train.mean()
std  = y_train.std()

y_train = (y_train - mean)/std
y_test  = (y_test - mean)/std

y_train = np.expand_dims(y_train, -1).astype("float32")
y_test  = np.expand_dims(y_test, -1).astype("float32")

n_lat, n_lon = y_train.shape[1], y_train.shape[2]
print(y_train.shape,y_test.shape)
#OUTPUT_DIM =

(753, 96, 144, 1) (86, 96, 144, 1)


In [9]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt

In [10]:
latent_dim = 2   # dimensions of the latent space
n_filters  = 32  # number of filters for the convolutional layers
n_neurons  = 32  # number of neurons for the Dense layers
activation = 'relu' # activation function
kernel_size = 4
learning_rate = 0.001
minibatch_size = 64
num_epochs     = 200

In [11]:
# Generator model
latent_dim = 100  # Set the desired dimension of the latent space
n_filters = 32  # Set the number of filters for the convolutional layers

decoder_input = Input(shape=(latent_dim,))
cond_input = Input(shape=(2,))

x = Concatenate(axis=1)([decoder_input, cond_input])
x = Dense(12 * 18 * n_filters, activation='relu')(x)
x = Reshape((12, 18, n_filters))(x)
x = Conv2DTranspose(n_filters, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
x = Conv2DTranspose(n_filters, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
x = Conv2DTranspose(n_filters, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
decoder_output = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

generator_model = Model([decoder_input, cond_input], decoder_output)

# Discriminator model
discriminator_input = Input(shape=(96, 144, 1))

x = Conv2D(n_filters, (3, 3), activation='relu', strides=(2, 2), padding='same')(discriminator_input)
x = Conv2D(n_filters, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
x = Conv2D(n_filters, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
x = Flatten()(x)
discriminator_output = Dense(1, activation='sigmoid')(x)

discriminator_model = Model(discriminator_input, discriminator_output)

# GAN model
gan_input = [decoder_input, cond_input]
gan_output = discriminator_model(generator_model(gan_input))

gan_model = Model(gan_input, gan_output)

# Compile discriminator model
discriminator_model.compile(loss='binary_crossentropy', optimizer='adam')

# Freeze discriminator weights during GAN training
discriminator_model.trainable = False

# Compile GAN model
gan_model.compile(loss='binary_crossentropy', optimizer='adam')

# Print summary of GAN model
gan_model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 model (Functional)             (None, 96, 144, 1)   739969      ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 model_1 (Functional)           (None, 1)            25729       ['model[0][0]']            

In [12]:
discriminator_inputs = Input(shape=(n_lat, n_lon, 1))  # shape: (96, 144, 1)
x = Conv2D(n_filters, kernel_size, activation=activation, strides=2, padding="same")(discriminator_inputs)  # shape: (48, 72, 32)
x = Conv2D(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x)  # shape: (24, 36, 32)
x = Conv2D(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x)  # shape: (12, 18, 32)
x = Flatten()(x)  # shape: (6912,)
x = Dense(n_neurons, activation=activation)(x)  # shape: (16,)
regression_output = Dense(1)(x)  # shape: (1,)

discriminator_model = Model(discriminator_inputs, regression_output, name="discriminator")
discriminator_model.summary()


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 144, 1)]      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 48, 72, 32)        544       
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 36, 32)        16416     
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 18, 32)        16416     
                                                                 
 flatten_1 (Flatten)         (None, 6912)              0         
                                                                 
 dense_2 (Dense)             (None, 32)                221216    
                                                                 
 dense_3 (Dense)             (None, 1)               

In [37]:
# Training loop
epochs = 100  # Set the number of training epochs
batch_size = 32  # Set the batch size for training

for epoch in range(epochs):
    print("Epoch:", epoch + 1)

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of real samples
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_samples = y_train[idx]
    real_labels = np.ones((batch_size, 1))

    # Generate a batch of fake samples
    latent_samples = np.random.normal(0, 1, (batch_size, latent_dim))
    cond_samples = X_train[idx]
    fake_samples = generator_model.predict([latent_samples, cond_samples])
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator
    discriminator_loss_real = discriminator_model.train_on_batch(real_samples, real_labels)
    discriminator_loss_fake = discriminator_model.train_on_batch(fake_samples, fake_labels)
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

    # -----------------
    #  Train Generator
    # -----------------

    # Generate a new batch of latent samples and random condition samples
    latent_samples = np.random.normal(0, 1, (batch_size, latent_dim))
    cond_samples = X_train[idx]
    generator_labels = np.ones((batch_size, 1))

    # Train the generator
    generator_loss = gan_model.train_on_batch([latent_samples, cond_samples], generator_labels)

    # Print the losses
    print("Discriminator loss:", discriminator_loss)
    print("Generator loss:", generator_loss)
    print()


Epoch: 1


ValueError: ignored

In [38]:
# Training loop
epochs = 100  # Set the number of training epochs
batch_size = 32  # Set the batch size for training

for epoch in range(epochs):
    print("Epoch:", epoch + 1)

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of real samples
    idx = np.random.randint(0, X_train_processed.shape[0], batch_size)
    real_samples = y_train_processed[idx]
    real_labels = np.ones((batch_size, 1))

    # Generate a batch of fake samples
    latent_samples = np.random.normal(0, 1, (batch_size, latent_dim))
    cond_samples = X_train_processed[idx]
    fake_samples = generator_model.predict([latent_samples, cond_samples])
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator

    discriminator_loss_real = discriminator_model.train_on_batch(real_samples, real_labels)
    discriminator_loss_fake = discriminator_model.train_on_batch(fake_samples, fake_labels)
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

    # -----------------
    #  Train Generator
    # -----------------

    # Generate a new batch of latent samples and random condition samples
    latent_samples = np.random.normal(0, 1, (batch_size, latent_dim))
    cond_samples = X_train_processed[idx]
    generator_labels = np.ones((batch_size, 1))

    # Train the generator
    generator_loss = gan_model.train_on_batch([latent_samples, cond_samples], generator_labels)

    # Print the losses
    print("Discriminator loss:", discriminator_loss)
    print("Generator loss:", generator_loss)
    print()

Epoch: 1


ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt


# Plot the losses
plt.plot(discriminator_loss, label='Discriminator Loss')
plt.plot( generator_loss, label='Generator Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [33]:
import matplotlib.pyplot as plt

# Lists to store discriminator and generator losses
discriminator_loss = []
generator_loss = []

# Training loop
epochs = 100
batch_size = 32

for epoch in range(epochs):
    print("Epoch:", epoch + 1)

    # ... Training code ...

    # Append the losses to the respective lists
    discriminator_losses.append(discriminator_loss)
    generator_losses.append(generator_loss)

    # Print the losses
    print("Discriminator loss:", discriminator_loss)
    print("Generator loss:", generator_loss)
    print()

# Create the epoch values (x-values)
epochs_values = range(1, epochs + 1)

# Repeat the scalar losses to match the dimension of epochs_values
discriminator_losses = [discriminator_loss] * epochs
generator_losses = [generator_loss] * epochs

# Plot the losses
plt.plot(epochs_values, discriminator_losses, label='Discriminator Loss')
plt.plot(epochs_values, generator_losses, label='Generator Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Epoch: 1


AttributeError: ignored

In [ ]:
decoder_input = Input(shape=(latent_dim,))  # shape: (2,)
cond_input = Input(shape=(X_train.shape[1],))  # shape: (2,)
x = Concatenate(axis=1)([decoder_input, cond_input])  # shape: (4,)
x = Dense(12 * 18 * n_filters, activation=activation)(x)  # shape: (6912,)
x = Reshape((12, 18, n_filters))(x)  # shape: (12, 18, 32)
x = Conv2DTranspose(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x)  # shape: (24, 36, 32)
x = Conv2DTranspose(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x)  # shape: (48, 72, 32)
x = Conv2DTranspose(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x)  # shape: (96, 144, 32)
decoder_output = Conv2DTranspose(1, kernel_size, activation="linear", padding="same")(x)  # shape: (96, 144, 1)

generator_model = Model([decoder_input, cond_input], decoder_output, name="generator")
generator_model.summary()


In [ ]:
generator_model = Model([decoder_input, cond_input], decoder_output)
discriminator_model = Model(discriminator_inputs, regression_output)

# Compile discriminator model
discriminator_model.compile(loss='binary_crossentropy', optimizer='adam')

# Freeze discriminator weights during GAN training
discriminator_model.trainable = False

# Combined GAN model
gan_input = [discriminator_inputs, cond_input]
gan_outputs = discriminator_model(generator_model(gan_input))
gan = Model(gan_input, gan_output)

# Compile GAN model
#gan.compile(loss='binary_crossentropy', optimizer='adam')


In [ ]:
# Discriminator model
discriminator_input = Input(shape=(n_lat, n_lon, 1))
discriminator_output = discriminator_model(discriminator_input)

# Combined GAN model
gan_input = [discriminator_inputs, cond_input]
gan_output = discriminator_model(generator)
gan = Model(gan_input, gan_output)

# Compile discriminator model
discriminator_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Freeze discriminator weights during GAN training
discriminator_model.trainable = False

# Compile GAN model
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Print summary of GAN model
gan.summary()

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
discriminator = Discriminator()

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        output = self.model(x)
        return output

generator = Generator()

In [ ]:
lr = 0.001
num_epochs = 300
loss_function = nn.BCELoss()

In [ ]:
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [ ]:
train_data_length = 1024
train_data = torch.zeros((train_data_length, 2))
train_data[:, 0] = 2 * math.pi * torch.rand(train_data_length)
train_data[:, 1] = torch.sin(train_data[:, 0])
train_labels = torch.zeros(train_data_length)
train_set = [
    (train_data[i], train_labels[i]) for i in range(train_data_length)
]

In [ ]:
a = tf.convert_to_tensor(X_train)

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    a, batch_size=batch_size, shuffle=True
)

In [23]:

latent_dim = 2   # dimensions of the latent space
n_filters  = 32  # number of filters for the convolutional layers
n_neurons  = 32  # number of neurons for the Dense layers
activation = 'relu' # activation function
kernal_size = 4
learning_rate = 0.001
batch_size = 64
n_epochs     = 200

In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras import Model

# Generator
#latent_dim = 2
#n_filters = 32
#kernal_size = 4
#n_neurons = 32

generator_inputs = Input(shape=(latent_dim,))
generator_cond_inputs = Input(shape=(X_train.shape[1],))

x = keras.layers.Concatenate(axis=1)([generator_inputs, generator_cond_inputs])
x = Dense(12 * 18 * n_filters, activation='relu')(x)
x = Reshape((12, 18, n_filters))(x)
x = Conv2DTranspose(n_filters, kernal_size, activation='relu', strides=2, padding='same')(x)
x = Conv2DTranspose(n_filters, kernal_size, activation='relu', strides=2, padding='same')(x)
x = Conv2DTranspose(n_filters, kernal_size, activation='relu', strides=2, padding='same')(x)
generator_outputs = Conv2DTranspose(1, kernal_size, activation='linear', padding='same')(x)

generator = Model([generator_inputs, generator_cond_inputs], generator_outputs, name='generator')
generator.summary()

# Discriminator
discriminator_inputs = Input(shape=(n_lat, n_lon, 1))

x = Conv2D(n_filters, kernal_size, activation='relu', strides=2, padding='same')(discriminator_inputs)
x = Conv2D(n_filters, kernal_size, activation='relu', strides=2, padding='same')(x)
x = Conv2D(n_filters, kernal_size, activation='relu', strides=2, padding='same')(x)
x = Flatten()(x)
x = Dense(n_neurons, activation='relu')(x)
discriminator_outputs = Dense(1, activation='sigmoid')(x)

discriminator = Model(discriminator_inputs, discriminator_outputs, name='discriminator')
discriminator.summary()

# Define the GAN
gan_inputs = [generator_inputs, generator_cond_inputs]
gan_outputs = discriminator(generator(gan_inputs))

gan = Model(gan_inputs, gan_outputs, name='gan')
gan.summary()

# Compile the models
generator.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')
discriminator.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='binary_crossentropy')
gan.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='binary_crossentropy')



Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 2)]          0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 2)]          0           []                               
                                                                                                  
 concatenate_4 (Concatenate)    (None, 4)            0           ['input_19[0][0]',               
                                                                  'input_20[0][0]']               
                                                                                                  
 dense_11 (Dense)               (None, 6912)         34560       ['concatenate_4[0][0]']  

In [ ]:
x_real = X_train
y_real = y_train
fake_samples = X_test
fake_labels = y_test
n_samples = 1000

In [ ]:
 # Generate fake samples
fake_inputs = np.random.normal(size=(n_samples, latent_dim))
fake_cond_inputs = np.random.normal(size=(n_samples, X_train.shape[1]))
fake_samples = generator.predict([fake_inputs, fake_cond_inputs])


In [ ]:
# Generate samples from true distribution
def generate_samples(n_samples):
    x = np.random.randn(n_samples)
    y = np.sin(3 * x) + np.random.normal(0, 0.1, n_samples)
    return x, y

# Generate random noise for generator input
def generate_noise(n_samples, latent_dim):
    return np.random.randn(n_samples, latent_dim)


In [25]:
for epoch in range(n_epochs):
    # Generate real samples
    x_real, y_real = generate_samples(n_samples)
     # Generate fake samples
    fake_inputs = np.random.normal(size=(n_samples, latent_dim))
    fake_cond_inputs = np.random.normal(size=(n_samples, X_train.shape[1]))
    fake_samples = generator.predict([fake_inputs, fake_cond_inputs])


    # Train discriminator
    discriminator_loss_real = discriminator.train_on_batch(x_real, np.ones((n_samples,)))
    noise = generate_noise(batch_size, latent_dim)
    x_fake = generator.predict(noise)
    discriminator_loss_fake = discriminator.train_on_batch(x_fake, np.zeros((batch_size,)))
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

    # Train generator
    noise = generate_noise(batch_size, latent_dim)
    generator_loss = gan.train_on_batch(noise, np.ones((batch_size,)))

NameError: ignored

In [ ]:
latent_dim = 2   # dimensions of the latent space
n_filters  = 32  # number of filters for the convolutional layers
n_neurons  = 32  # number of neurons for the Dense layers
activation = 'relu' # activation function
kernal_size = 4
learning_rate = 0.001
minibatch_size = 64
num_epochs     = 100

In [ ]:
encoder_inputs = Input(shape=(n_lat, n_lon, 1)) # shape: (96,144,1)
x = Conv2D(n_filters, kernal_size, activation=activation, strides=2, padding="same")(encoder_inputs) # shape: (48,72,32)
x = Conv2D(n_filters, kernal_size, activation=activation, strides=2, padding="same")(x) # shape: (24,36,32)
x = Conv2D(n_filters, kernal_size, activation=activation, strides=2, padding="same")(x) # shape: (12,18,32)
x = Flatten()(x) # shape: (6912,1)
x = Dense(n_neurons, activation=activation)(x) # shape: (16,1)


z_mean    = Dense(latent_dim, name="z_mean")(x) # shape: (2,1)
z_log_var = Dense(latent_dim, name="z_log_var")(x) # shape: (2,1)


def sample_latent_features(distribution):
    """
    This function takes previously calculated mean & variance,
    and returns back a latent encoding vector.
    """

    distribution_mean, distribution_variance = distribution
    batch_size = tf.shape(distribution_variance)[0]
    random = tf.keras.backend.random_normal(shape=(batch_size, tf.shape(distribution_variance)[1]))
    return distribution_mean + tf.exp(0.5 * distribution_variance) * random


z         = Lambda(sample_latent_features)([z_mean, z_log_var])

encoder_model = Model(encoder_inputs, z, name="encoder")
encoder_model.summary()

In [ ]:
decoder_input = Input(shape=(latent_dim,)) # shape: (2,1)
cond_input    = Input(shape=(X_train.shape[1],)) # shape: (2,1)
x = keras.layers.Concatenate(axis=1)([decoder_input]) # shape: (4,1)
x = Dense(12 * 18 * n_filters, activation=activation)(x) # shape: (6912,1)
x = Reshape((12, 18, n_filters))(x) # shape: (12,18,32)
x = Conv2DTranspose(n_filters, kernal_size, activation=activation, strides=2, padding="same")(x) # shape: (24,36,32)
x = Conv2DTranspose(n_filters, kernal_size, activation=activation, strides=2, padding="same")(x) # shape: (48,72,32)
x = Conv2DTranspose(n_filters, kernal_size, activation=activation, strides=2, padding="same")(x) # shape: (96,144,32)
decoder_output = Conv2DTranspose(1, kernal_size, activation="linear", padding="same")(x) # shape: (96,144,1)
decoder_model = Model([decoder_input,cond_input], decoder_output, name="decoder")
decoder_model.summary()

In [17]:
n_filters = 32  # Number of filters in convolutional layers
kernel_size = (3, 3)  # Kernel size for convolutional layers
activation = "relu"  # Activation function for convolutional and dense layers
n_neurons = 16  # Number of neurons in the dense layer
latent_dim = 2  # Dimensionality of the latent space

# Example of X_train.shape for cond_input
X_train_shape = (2, 1)  # Replace with the appropriate shape of your training data



In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Reshape, Conv2DTranspose, Lambda
from tensorflow.keras.models import Model
import numpy as np

# Encoder
encoder_inputs = Input(shape=(n_lat, n_lon, 1)) # shape: (96, 144, 1)
x = Conv2D(n_filters, kernel_size, activation=activation, strides=2, padding="same")(encoder_inputs) # shape: (48, 72, 32)
x = Conv2D(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x) # shape: (24, 36, 32)
x = Conv2D(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x) # shape: (12, 18, 32)
x = Flatten()(x) # shape: (6912,)
x = Dense(n_neurons, activation=activation)(x) # shape: (16,)

z_mean = Dense(latent_dim, name="z_mean")(x) # shape: (2,)
z_log_var = Dense(latent_dim, name="z_log_var")(x) # shape: (2,)

def sample_latent_features(distribution):
    distribution_mean, distribution_variance = distribution
    batch_size = tf.shape(distribution_variance)[0]
    random = tf.keras.backend.random_normal(shape=(batch_size, tf.shape(distribution_variance)[1]))
    return distribution_mean + tf.exp(0.5 * distribution_variance) * random

z = Lambda(sample_latent_features)([z_mean, z_log_var])

encoder_model = Model(encoder_inputs, z, name="encoder")
encoder_model.summary()


# Decoder
decoder_input = Input(shape=(latent_dim,)) # shape: (2,)
cond_input = Input(shape=(X_train.shape[1],)) # shape: (2,)
x = keras.layers.Concatenate(axis=1)([decoder_input, cond_input]) # shape: (4,)
x = Dense(12 * 18 * n_filters, activation=activation)(x) # shape: (6912,)
x = Reshape((12, 18, n_filters))(x) # shape: (12, 18, 32)
x = Conv2DTranspose(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x) # shape: (24, 36, 32)
x = Conv2DTranspose(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x) # shape: (48, 72, 32)
x = Conv2DTranspose(n_filters, kernel_size, activation=activation, strides=2, padding="same")(x) # shape: (96, 144, 32)
decoder_output = Conv2DTranspose(1, kernel_size, activation="linear", padding="same")(x) # shape: (96, 144, 1)
decoder_model = Model([decoder_input, cond_input], decoder_output, name="decoder")
decoder_model.summary()


# GAN (Generator + Discriminator)
gan_input = Input(shape=(latent_dim,))
gan_cond_input = Input(shape=(X_train.shape[1],))
generated_image = decoder_model([gan_input, gan_cond_input])

# Freeze the weights of the encoder during GAN training
encoder_model.trainable = False

# Discriminator
discriminator_input = Input(shape=(n_lat, n_lon, 1))
discriminator_features = encoder_model(discriminator_input)
discriminator_output = Dense(1, activation="sigmoid")(discriminator_features)

discriminator_model = Model(discriminator_input, discriminator_output, name="discriminator")
discriminator_model.summary()

# Compile the discriminator
discriminator_model.compile(optimizer=keras.optimizers.Adam(), loss="binary_crossentropy")

# Combined GAN model (Generator + Discriminator)
gan_output = discriminator_model(generated_image)
gan = Model([gan_input, gan_cond_input], gan_output)
gan.summary()

# Compile the GAN
gan.compile(optimizer=keras.optimizers.Adam(), loss="binary_crossentropy")


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 96, 144, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_10 (Conv2D)             (None, 48, 72, 32)   320         ['input_11[0][0]']               
                                                                                                  
 conv2d_11 (Conv2D)             (None, 24, 36, 32)   9248        ['conv2d_10[0][0]']              
                                                                                                  
 conv2d_12 (Conv2D)             (None, 12, 18, 32)   9248        ['conv2d_11[0][0]']        